In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bertopic import BERTopic

from nltk import word_tokenize
import os.path as op
import re 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [128]:
# import the preprocess doc

df = pd.read_csv('./Data_csv/data_preprocessed.csv')

In [129]:
df.columns

Index(['doc_id', 'Name of the document', 'Institution', 'URL', 'Authors',
       'Affiliates', 'Sector', 'Country', 'Date', 'Keywords',
       'Exclusion criteria', 'Status', 'Label', 'MapAIE (ours)', 'Jobin',
       'Fjeld', 'Tidjon', 'Hagendorff', 'Floridi', 'Zeng (LAIP)',
       'Attard-Frost', 'EP', 'Algorithm watch', 'CE', 'Winfield',
       'EthicalML GitHub', 'all sources', 'Checked by', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'text', 'langue', 'text_processed',
       'tfidf', 'categorie Institution', 'theme'],
      dtype='object')

In [130]:
df.count()

doc_id                   458
Name of the document     458
Institution              458
URL                      458
Authors                   33
Affiliates                28
Sector                    43
Country                  446
Date                      45
Keywords                   6
Exclusion criteria         0
Status                   458
Label                    394
MapAIE (ours)            458
Jobin                     61
Fjeld                     20
Tidjon                    22
Hagendorff                12
Floridi                    3
Zeng (LAIP)               43
Attard-Frost              28
EP                         8
Algorithm watch          104
CE                       378
Winfield                  15
EthicalML GitHub          12
all sources              458
Checked by               458
Unnamed: 27                2
Unnamed: 28                0
Unnamed: 29                2
text                     458
langue                   458
text_processed           458
tfidf         

In [131]:
df['text_processed'][0]

"intel recommends public policy principles artificial intelligence intel community longer support internet explorer v10 older compatibility view enabled disable compatibility view upgrade newer version use different browser search browse support community developer software forums developer software forums software development tools toolkits sdks software development topics software development technologies oneapi registration download licensing installation gpu compute software intel® tiber developer cloud software archive edge software catalog product support forums product support forums fpga memory storage visual computing embedded products graphics processors wireless ethernet products server products intel vpro® platform intel® enpirion® power solutions intel® unison™ app intel® quickassist technology intel® qat intel® trusted execution technology intel® txt thunderbolt™ share intel® gaudi® ai accelerator gaming forums gaming forums intel® arc™ graphics gaming intel® processors i

In [132]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=15, min_samples=1, cluster_selection_epsilon=0.1)
topic_model = BERTopic(hdbscan_model=hdbscan_model, language="english", nr_topics=10)


In [133]:
documents = df['text_processed'].to_list()

In [134]:
vectorizer = CountVectorizer(stop_words='english')
Bow = vectorizer.fit_transform(documents)

# Calculer la fréquence totale de chaque mot
frequency = Bow.toarray().sum(axis=0)

# Obtenir les 20 mots les plus fréquents
top_indices = np.argsort(frequency)[-20:]  # Indices des 20 mots les plus fréquents
vocabulary = np.array(vectorizer.get_feature_names_out())  # Le vocabulaire
top_words = vocabulary[top_indices] 

stopwords_to_remove = set(top_words)

dates = set([str(i) for i in range(1990, 2021)])
html_stopwords = set({'maxwidth', 'px', 'maxheight', 'img', 'src', 'https', 'com', 'www', 'http', 'jpg', 'png', 'gif', 'jpeg', 'pdf', 'html'})
css_stopwords = set({'grid','body', ':root', '\n', 'padding',',button','margintop', 'px', 'margin', 'border', 'width', 'height', 'color', 'font', 'size', 'background', 'position', 'left', 'right', 'top', 'bottom', 'display', 'flex', 'align', 'justify', 'content', 'center', 'float', 'clear', 'overflow', 'hidden', 'zindex', 'cursor', 'pointer', 'hover', 'active', 'focus', 'transition', 'transform', 'rotate', 'scale', 'translate', 'opacity', 'box', 'shadow', 'outline', 'none', 'block', 'inline', 'inlineblock', 'relative', 'absolute', 'fixed', 'static', 'sticky', 'visible', 'hidden'})

stopwords_to_remove = stopwords_to_remove.union(dates)
stopwords_to_remove = stopwords_to_remove.union(html_stopwords)
stopwords_to_remove = stopwords_to_remove.union(css_stopwords)




def remove_frequent_words(doc, stopwords):
    words = doc.split()
    cleaned_words = [word for word in words if word not in stopwords]
    return " ".join(cleaned_words)


documents = [remove_frequent_words(doc, stopwords_to_remove) for doc in documents]



In [135]:
topics, probs = topic_model.fit_transform(documents)

In [136]:
df['topic'] = topics
df['probs'] = probs

In [137]:
# Afficher les topics les plus fréquents
print(topic_model.get_topic_info())



   Topic  Count                                             Name  \
0     -1     65                -1_media_maxwidth_important_767px   
1      0     34                     0_system_also_may_protection   
2      1     31              1_uk_government_strategy_innovation   
3      2     72                    2_algorithmic_system_may_bias   
4      3     95                  3_ethics_ethical_machine_system   
5      4     40  4_government_international_services_cooperation   
6      5    121                          5_eu_also_system_policy   

                                      Representation  \
0  [media, maxwidth, important, 767px, margintop,...   
1  [system, also, may, protection, used, personal...   
2  [uk, government, strategy, innovation, work, s...   
3  [algorithmic, system, may, bias, algorithms, m...   
4  [ethics, ethical, machine, system, may, also, ...   
5  [government, international, services, cooperat...   
6  [eu, also, system, policy, national, based, la...   

      

In [138]:
print(topic_model.get_topic(-1)) 

[('media', 0.030944214109267166), ('maxwidth', 0.029436405774000654), ('important', 0.02523083124878923), ('767px', 0.018624442927137802), ('margintop', 0.01567078684202728), ('100', 0.015610790783711952), ('backgroundcolor', 0.01476998524118875), ('fontsize', 0.014619967134584543), ('16px', 0.014536464382859304), ('24px', 0.010940878853319038)]


In [139]:
# Visualisation des topics
topic_model.visualize_topics()


In [140]:

# Visualisation des hiérarchies entre les topics
topic_model.visualize_hierarchy()

In [141]:

# Visualisation des relations entre les topics
topic_model.visualize_barchart()

In [142]:
topic_model.visualize_heatmap(n_clusters=8)

ValueError: Make sure to set `n_clusters` lower than the total number of unique topics.

In [ ]:
topic_model.get_representative_docs(-1)

["october national strategic plan national science council networking subcommittee ii iii national science council national science council nstc principal means executive branch coordinates science policy across diverse entities make federal r enterprise one nstc ’ primary objectives establishing national goal federal science investments nstc prepares r packages aimed accomplishing multiple national goals nstc ’ work organized five committees environment natural resources sustainability homeland national ecurity science engineering mathematics stem education science committees oversees subcommittees working groups focused different aspects science available www.whitehouse.gov/ostp/nstc office science policy office science policy ostp established national science policy organization priorities act 1976 mission ostp threefold first provide president senior staff accurate relevant timely scientific technical advice matters consequence second ensure th policies executive branch informed so

In [ ]:
df = pd.DataFrame({"Document": docs, "Topic": topic})

NameError: name 'docs' is not defined

In [ ]:
df[df["topic"] == -1]

,doc_id,Name of the document,Institution,URL,Authors,Affiliates,Sector,Country,Date,Keywords,...,Unnamed: 28,Unnamed: 29,text,langue,text_processed,tfidf,categorie Institution,theme,topic,probs
3,13,The Toronto Declaration: Protecting the right ...,Access Now,https://www.torontodeclaration.org/declaration...,"Anna Bacciarelli, Joe Westby, Fanny Hidvegi, E...","Access Now, AI Now Institute at New York Unive...",civil society,UK,2018-05-16,NaN,...,NaN,NaN,\n\n\n\n\nThe Toronto Declaration\n\n\n\n\n\n\...,en,toronto declaration skip main content toronto ...,[0. 0. 0. ... 0. 0. 0.],ONG et initiatives de droits humains,Justice and fairness,-1,0.0
16,29,AI Now Report 2016. The Social and Economic Im...,AI Now Institute,https://artificialintelligencenow.com/media/do...,NaN,NaN,NaN,USA,NaN,NaN,...,NaN,NaN,\n :root {\n --small-font: 12px;\n ...,en,root -- small-font 12px -- normal-font 12px --...,[0. 0.00579831 0. ... 0. ...,Instituts de recherche et universités,Transparency,-1,0.0
32,49,It’s Time to Do Something: Mitigating the Nega...,Association for Computing Machinery - Future o...,https://perma.cc/K22T-5DFU,NaN,NaN,NaN,USA,NaN,NaN,...,NaN,NaN,\n\nPerma | It’s Time to Do Something: Mitigat...,en,perma ’ time something mitigating negative imp...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Transparency,-1,0.0
33,51,"Statement on Accessibility, Usability, and Dig...",Association for Computing Machinery - Future o...,https://www.acm.org/binaries/content/assets/pu...,NaN,NaN,NaN,USA,NaN,NaN,...,NaN,NaN,ACM US Public Policy Council 1701 Pennsylv...,en,acm us public policy council 1701 pennsylvania...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Justice and fairness,-1,0.0
34,52,Statement on Computing and Network Security,Association for Computing Machinery - Future o...,https://www.acm.org/binaries/content/assets/pu...,NaN,NaN,NaN,EU,NaN,NaN,...,NaN,NaN,ACM US Public Policy Council 1701 Pennsylv...,en,acm us public policy council 1701 pennsylvania...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Non-maleficence,-1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,718,AI procurement in a box: AI government procure...,World Economic Forum (WEF),https://www3.weforum.org/docs/WEF_AI_Procureme...,NaN,NaN,NaN,international,NaN,NaN,...,NaN,NaN,AI Procurement in a Box: Project overviewUnl...,en,ai procurement box project overviewunlocking p...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Trust,-1,0.0
450,719,Artificial Intelligence for Children - Toolkit,World Economic Forum (WEF),https://www3.weforum.org/docs/WEF_Artificial_I...,NaN,NaN,NaN,international,NaN,NaN,...,NaN,NaN,AI Procurement in a Box: Project overviewUnl...,en,ai procurement box project overviewunlocking p...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Trust,-1,0.0
451,720,Guidelines for AI procurement,Department for Digital,https://www.gov.uk/government/publications/gui...,NaN,NaN,NaN,UK,NaN,NaN,...,NaN,NaN,White Paper Guidelines for AI Procurement Sept...,en,white paper guidelines ai procurement septembe...,[0. 0.0077954 0. ... 0. 0...,Gouvernements et organismes publics nationaux,Responsibility,-1,0.0
452,721,How to prevent Discriminatory Outcomes in Mach...,World Economic Forum (WEF),http://www3.weforum.org/docs/WEF_40065_White_P...,NaN,NaN,NaN,international,NaN,NaN,...,NaN,NaN,White Paper Guidelines for AI Procurement Sept...,en,white paper guidelines ai procurement septembe...,[0. 0.0077954 0. ... 0. 0...,Associations professionnelles et groupes de ré...,Responsibility,-1,0.0


In [ ]:

df[df['doc_id']==29]['text']

16    \n    :root {\n      --small-font: 12px;\n    ...
Name: text, dtype: object